# CS/ECE/ISyE 524 - Spr 2021 - HW 9
### Huong Thien Do

# 1. Making Change

How should you make change for 99 cents if the goal is to minimize the total weight of the coins used? You may use any number of each type of coin. Here are the weights of each coin:

|Type of Coin       | penny | nickel | dime | quarter |
|:------------------|-------|--------|------|---------|
|Weight             |  2.500|5.000   | 2.268| 5.670   |



Print the number of each type of coin used, and the total weight.

In [ ]:
# problem data
weights = [ 2.500, 5.000, 2.268, 5.670]
values = [ 1, 5, 10, 25 ]
n = length(weights) # total number of coin types


In [1]:
using Pkg
Pkg.add("JuMP")
Pkg.add("HiGHS")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [6]:
using JuMP, HiGHS

m = Model(HiGHS.Optimizer)
set_silent(m)
@variable(m, penny >= 0, Int)
@variable(m, nickel >= 0, Int)
@variable(m, dime >= 0, Int)
@variable(m, quarter >= 0, Int)

@constraint(m, penny + nickel*5 + dime*10 + quarter*25 == 99)

@objective(m, Min, penny*2.5 + nickel*5 + dime*2.268 + quarter*5.67)

optimize!(m)

println("Penny = ", value.(penny) )
println("Nickel = ", value.(nickel) )
println("Dime = ", value.(dime) )
println("Quarter = ", value.(quarter) )
println("The total weight is ", objective_value(m), " grams." )

Penny = 4.0
Nickel = 0.0
Dime = 2.0
Quarter = 3.0
The total weight is 31.546 grams.


# 2. Comquat Computers

Comquat owns four production plants at which personal computers are produced. Comquat can sell up to 20,000 computers per year at a price of \$3,500 per computer. For each plant the production capacity, cost per computer, and fixed cost of operating the plant for a year are given below. Determine how Comquat can maximize its yearly profit from computer production.

| Plant | Production capacity | Plant fixed cost (\$ Million) | Cost per computer (\$) |
|:-----:|:-------------------:|:-----------------------------:|:----------------------:|
| 1     | 10,000              |   9                           |  1,000                 |
| 2     |  8,000              |   5                           |  1,700                 |
| 3     |  9,000              |   3                           |  2,300                 |
| 4     |  6,000              |   1                           |  2,900                 |

In [10]:
using JuMP, HiGHS
m = Model(HiGHS.Optimizer)
set_silent(m)

price     = 3.5e3                       # price at which we can sell a single computer 
                                        # (regardless of where it is produced)
cap       = 20000                       # maximum computers that can be sold per year

capacity  = [10000, 8000, 9000, 6000]   # production capacity for each plant
fixedcost = [9e6, 5e6, 3e6, 1e6]        # fixed cost for each plant
itemcost  = [1e3, 1.7e3, 2.3e3, 2.9e3]  # cost per computer for each factory


# insert your code here, with integer variable vector z and real variable vector x
@variable(m, 0 <= x[1:4], Int)  # number of computers produced by each plant.
@variable(m, z[1:4], Bin)       # binary variable.

for i in 1:length(capacity)
    @constraint(m, x[i] <= capacity[i]*z[i]) # production capacity 
end

@constraint(m, sum(x) <= cap) # total production capacity

for i in 1:length(capacity)
    @constraint(m, x[i] <= z[i]*20000) # if each factory is open
end

@objective(m, Max, sum((price - itemcost[i])* x[i] - fixedcost[i]*z[i] for i in 1:length(capacity)))

optimize!(m)
println("The maximal profit is: ", objective_value(m))
println("Here is a tally of factory number, whether it's open or not, and how many computers it should produce.")
value.([1:4 z x])

The maximal profit is: 2.56e7
Here is a tally of factory number, whether it's open or not, and how many computers it should produce.


4×3 Matrix{Float64}:
 1.0  1.0  10000.0
 2.0  1.0   8000.0
 3.0  0.0      0.0
 4.0  1.0   2000.0

# 3. ABC Investments

ABC Inc. is considering several investment options.  Each option has a minimum investment required as well as a maximum investment allowed. These restrictions, along with the expected return are summarized in the following table (figures are in millions of dollars):

|Option | Minimum investment | Maximum investment | Expected return (%)  |
|:-----:|:------------------:|:------------------:|:--------------------:|
|  1    |      3             |      27            |     13               |
|  2    |      2             |      12            |      9               |
|  3    |      9             |      35            |     17               |
|  4    |      5             |      15            |     10               |
|  5    |     12             |      46            |     22               |
|  6    |      4             |      18            |     12               |

Because of the high-risk nature of Option 5, company policy requires that the total amount invested in Option 5 be no more that the combined amount invested in Options 2, 4 and 6.  In addition, if an investment is made in Option 3, it is required that at least a minimum investment be made in Option 6.  ABC has \$80 million to invest and obviously wants to maximize its total expected return on investment.  Which options should ABC invest in, and how much should be invested?

In [11]:
using JuMP, HiGHS
m = Model(HiGHS.Optimizer)
set_silent(m)

lows  = [  3,  2,  9,  5, 12,  4 ]  # minimum investment
highs = [ 27, 12, 35, 15, 46, 18 ]  # maximum investment
ret   = [ 13,  9, 17, 10, 22, 12 ]  # expected return

# your code here
n = length(ret)

@variable(m, z[1:n], Bin)
@variable(m, x[1:n] >= 0)

for i=1:n
    @constraint(m, x[i] <= highs[i]*z[i])
    @constraint(m, x[i] >= lows[i]*z[i])
end

@constraint(m, x[2]+x[4]+x[6] >= x[5])
@constraint(m, sum(x) <= 80)
@constraint(m, lows[6]*z[3] <=  x[6])

@objective(m, Max, sum((x[i] * ret[i])/100 for i = 1:n))
optimize!(m)
    
println("The maximum return on investment (net profit) is: \$", objective_value(m), " million.")
println("This is an average return of: ", 100*objective_value(m)/value(sum(x)), " %")
println("Here is a tally of each investment, whether we invest or not, and how much we invest (in millions of \$).")
value.([1:6 z x])

The maximum return on investment (net profit) is: $13.5 million.
This is an average return of: 16.875 %
Here is a tally of each investment, whether we invest or not, and how much we invest (in millions of $).


6×3 Matrix{Float64}:
 1.0  0.0   0.0
 2.0  0.0   0.0
 3.0  1.0  35.0
 4.0  1.0   5.0
 5.0  1.0  22.5
 6.0  1.0  17.5

# 4. Paint production

As part of its weekly production, a paint company produces five batches of paints, always the same, for some big clients who have a stable demand.  Every paint batch is produced in a single production process, all in the same blender that needs to be cleaned between each batch.  The durations of blending paint batches 1 to 5 are 40, 35, 45, 32 and 50 minutes respectively.  The cleaning times depend of the colors and the paint types.  For example, a long cleaning period is required if an oil-based paint is produced after a water-based paint, or to produce white paint after a dark color.  The times are given in minutes in the following matrix $A$ where $A_{ij}$ denotes the cleaning time after batch $i$ if it is followed by batch $j$.

$$
A = \begin{bmatrix}
 0&11& 7&13&11 \\
 5& 0&13&15&15 \\
13&15& 0&23&11 \\
 9&13& 5& 0& 3 \\
 3& 7& 7& 7& 0
\end{bmatrix}
$$

Since the company has other activities, it wishes to deal with this weekly production in the shortest possible time (blending and cleaning).  What is the corresponding order of paint batches?  The order will be applied every week, so the cleaning time between the last batch of one week and the first of the following week needs to be accounted for in the total duration of cleaning.
                

In [13]:
# A[i,j] is the time it takes to clean after batch i if followed by batch j
A = [  0  11   7  13  11
       5   0  13  15  15
      13  15   0  23  11
       9  13   5   0   3
       3   7   7   7   0 ]

# time it takes to blend a particular batch. 
b = [ 40, 35, 45, 32, 50 ]

using JuMP, HiGHS, LinearAlgebra
m = Model(HiGHS.Optimizer)
set_silent(m)

n = length(b)  # number of paints

# YOUR CODE HERE
@variable(m, 0 <= x[1:n, 1:n], Int)

for i = 1:n
    @constraint(m, sum(x[i, :]) <= 1)
    @constraint(m, sum(x[:, i]) <= 1)
    @constraint(m, x[i, i] == 0)
end

for i in 1:n, j in 1:n
    @constraint(m, x[i, j] + x[j, i] <= 1)
end
@constraint(m, sum(x) == 5)

@objective(m, Min, dot(vec(A), vec(x)))

optimize!(m)

X = value.(x);
for i = 1:n
    for j=1:n
        if (X[i,j] >= .99)
            println(" blend ",i," is followed by blend ",j," requiring cleanup time ",A[i,j])
        end
    end
end

println("and the minimum cleanup time is: ", objective_value(m), " min, (plus ", sum(b), " min of blending)")

 blend 1 is followed by blend 4 requiring cleanup time 13
 blend 2 is followed by blend 1 requiring cleanup time 5
 blend 3 is followed by blend 5 requiring cleanup time 11
 blend 4 is followed by blend 3 requiring cleanup time 5
 blend 5 is followed by blend 2 requiring cleanup time 7
and the minimum cleanup time is: 41.0 min, (plus 202 min of blending)
